#### Example notebook on how to use _firstDown_

In [1]:
import firstDown

# Load data
pbp, players = firstDown.load_data.datasets.nfl_data()

# Get players' positions
dataset = firstDown.feature_engineering.build_features.get_positions(pbp, players)

# Clean data
dataset = firstDown.preprocessing.clean.drop_penalties(dataset, penalty_col='first_down_penalty')
dataset = firstDown.preprocessing.clean.drop_control_rows(dataset, control_col='desc')

In [2]:
# Search for NaN values
nan = firstDown.preprocessing.deal_nan.search_nan(dataset)
display(nan)

,var,rows_nan,rows_other,rows_total
0,play_id,0,36228,36228
1,game_id,0,36228,36228
2,old_game_id,0,36228,36228
3,home_team,0,36228,36228
4,away_team,0,36228,36228
...,...,...,...,...
367,xpass,8051,28177,36228
368,pass_oe,8825,27403,36228
369,rush_pos,24404,11824,36228
370,rec_pos,22509,13719,36228


In [3]:
# If needed, replace or drop NaN values using replace_nan or drop_nan

In [4]:
# Feature Engineering
dataset = firstDown.feature_engineering.build_features.inertia(dataset)
dataset = firstDown.feature_engineering.build_features.play_type(dataset)
dataset = firstDown.feature_engineering.build_features.defense_rush(dataset)
dataset = firstDown.feature_engineering.build_features.defense_pass(dataset)
dataset = firstDown.feature_engineering.build_features.defense_scramble(dataset)

# Select relevant features
dataset = dataset[['first_down','ydstogo','down','inertia','score_differential','play_category','rush_pos','pass_pos','rec_pos','def_vs_rush','def_vs_pass','def_vs_qb_scramble','shotgun','wp','temp','wind','roof','surface','location','half_seconds_remaining','game_half','yardline_100']]

In [5]:
# Train / test split
X_train, X_test, y_train, y_test = firstDown.preprocessing.split.split_data(dataset, 'first_down', 0.2)

In [6]:
# Fill nan for temp and wind

X_train, value_w = firstDown.preprocessing.deal_nan.replace_nan(X_train, 'wind', method='num')
X_train, value_t = firstDown.preprocessing.deal_nan.replace_nan(X_train, 'temp', method='median')

X_test, value_w = firstDown.preprocessing.deal_nan.replace_nan(X_test, 'wind', method='num')
X_test, value_t = firstDown.preprocessing.deal_nan.replace_nan(X_test, 'temp', method='num', num=value_t)

In [7]:
# Scale numerical columns
num_cols = ['ydstogo','inertia','score_differential','def_vs_rush','def_vs_pass','def_vs_qb_scramble','wp','temp','wind','half_seconds_remaining','yardline_100']

X_train = firstDown.preprocessing.scale.scaler(X_train, num_cols=num_cols)
X_test = firstDown.preprocessing.scale.scaler(X_test, num_cols=num_cols)

# Encoding columns
one_hot_cols = ['down','play_category','rush_pos','pass_pos','rec_pos','roof','surface','location','game_half']

encoder = firstDown.feature_engineering.encode.one_hot(X_train, one_hot_cols)
X_train = firstDown.feature_engineering.encode.one_hot_transform(X_train, one_hot_cols, encoder)
X_test = firstDown.feature_engineering.encode.one_hot_transform(X_test, one_hot_cols, encoder)

In [8]:
# Fit Model
clf = firstDown.train.models.do_fit(X_train, y_train)
y_pred, y_pred_prob = firstDown.train.models.do_predict(X_test,clf)

[LightGBM] [Info] Number of positive: 6258, number of negative: 22724
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005680 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1472
[LightGBM] [Info] Number of data points in the train set: 28982, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215927 -> initscore=-1.289561
[LightGBM] [Info] Start training from score -1.289561


In [9]:
# Model performance metrics
metrics = {
        "Accuracy": firstDown.metrics.model_metrics.accuracy(y_test, y_pred),
        "Recall": firstDown.metrics.model_metrics.recall(y_test, y_pred),
        "Precision": firstDown.metrics.model_metrics.precision(y_test, y_pred),
        "F1 Score": firstDown.metrics.model_metrics.f1(y_test, y_pred),
        "ROC AUC Score": firstDown.metrics.model_metrics.roc_auc(y_test, y_pred_prob)
}

print(metrics)

{'Accuracy': 0.8193486061275186, 'Recall': 0.3579920739762219, 'Precision': 0.6166097838452788, 'F1 Score': 0.4529878813205182, 'ROC AUC Score': 0.8349320623241004}
